In [11]:
import os
import json
import lxml
from bs4 import BeautifulSoup as bs
from pathlib import Path
import sys
import json
import csv
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azureml.core import Workspace, Dataset,Datastore
from azureml.data.datapath import DataPath
import ast
import re
import numpy as np
from pathlib import Path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from computer_version import computer_version_rules
from form_recognizer import doc_read
from rules import fraud_document_analytics

In [12]:
datastore_name = "dl_aalab_mlworkspace_gbpii"
ws = Workspace.from_config()
datastore = Datastore(ws, datastore_name)
input_name ='cdn-aa-gb-fraud-doc-analytics/OCS Files'
# input_name ='cdn-aa-gb-fraud-doc-analytics/test'
input_dataset = Dataset.File.from_files(path = [(datastore,input_name)])
input_mount_ctx = input_dataset.mount()  
input_mount_ctx.start()
input_dataset_mount_folder = input_mount_ctx.mount_point
input_path = input_dataset_mount_folder+'/'
local_output_path = 'output/'

fuse: warning: library too old, some operations may not not work


In [8]:
cv = computer_version_rules(input_path,local_output_path)

In [7]:
date_name = 'Test'
batch_name = '111'
test = cv.detect_stamp(date_name,batch_name,input_path)

In [8]:
test

[]

In [9]:
files_stamp_detected = []
for datename in os.listdir(input_path):
    date_path = input_path+datename+'/'
    for batchname in os.listdir(date_path):
        batch_path = date_path+batchname+'/'
        files = cv.detect_stamp(datename,batchname,batch_path)
        files_stamp_detected = files_stamp_detected + files
        

NotADirectoryError: [Errno 20] Not a directory: '/tmp/tmp7miwt6bx/Part_1/0050064374852_Step-_Element-_34e24519-5398-4a8d-9554-f8ba08da5e82_IMG_4909.JPEG.jpg/'

In [6]:
test

['Test_111_00030.tif']

In [9]:
fr = doc_read(input_path,local_output_path)

In [6]:
date_name = 'Test'
batch_name = '111'
input_path = 'output/'
files_with_stamp = cv.detect_stamp(date_name,batch_name,input_path)
test2  = fr.read_document(date_name,batch_name,input_path)
test2  = fr.sort_document(batch_name, test2)
# test2  = fr.parse_metadata(batch_name, input_path,local_output_path, test2)
test2 = fr.handwritten_per(batch_name,test2)
test2 = fr.check_stamp(test2,files_with_stamp)

Read 1 images' text in batch 111
Labelled batch 111
Calculating handwritten percentage of files in batch 111


In [7]:
test2 = rules.paid_by_cash(test2)
test2 = rules.has_dollar_symbol(test2)
test2 = rules.handwritten_check(test2,0.3)
test2 = rules.invoice_num_check(test2)
test2 = rules.get_payment(test2)
test2 = rules.address_check(test2)
test2 = rules.get_reg_num(test2)

In [8]:
test2

,file_name,content,word_confidence,styles,page_label,handwritten_percentage,has_stamp,paid_by_cash,no_dollar_symbol,above_handwritten_threshold,invalid_invoice_nbr,possible_invoice_nbr,payment_amount,has_invalid_addr,verified_address,register_num
0,Test_111_Capture2.jpg,THE PHYSIO SPOT HEALTH CLINIC INVOICE TO: NANC...,"{'THE': 0.997, 'PHYSIO': 0.995, 'SPOT': 0.991,...",[],paramedical invoice,0,False,False,True,False,False,None,None,True,[],None


In [7]:
process_list = ['Oct-27-2022','Oct-31-2022']

In [5]:
for datename in os.listdir(input_path):
    if datename in process_list:
        date_path = input_path+datename+'/'

        for batchname in os.listdir(date_path):
            if batchname in ['08000013-155028','08000014-155028', '08000015-160242'] or datename == 'Oct-31-2022':
                print(datename)
                print(batchname)
                batch_path = date_path + batchname +'/'
                files_with_stamp = cv.detect_stamp(datename,batchname,batch_path)
                df_out = fr.read_document(datename,batchname,batch_path)
                df_out = fr.sort_document(batchname, df_out)
                # df_out = fr.parse_metadata(batchname, batch_path,local_output_path, df_out)
                df_out = fr.handwritten_per(batchname,df_out)
                df_out = fr.check_stamp(df_out, files_with_stamp)



                # write to a tmp path on compute instance
                temp_out_file_path = local_output_path + datename + '_'+batchname + '.csv'
                df_out.to_csv(temp_out_file_path, index = False)

                print('read and label batch {} successfully, save to {}'.format(batchname,datename + '_'+batchname + '.csv'))

NameError: name 'process_list' is not defined

In [10]:
# For OCS files test 

for part in os.listdir(input_path):    
    batch_path = input_path + part +'/'
    files_with_stamp = cv.detect_stamp('OCS_files_2',part,batch_path)
    df_out = fr.read_document('OCS_files_2',part,batch_path)
    df_out = fr.sort_document(part, df_out)
    # df_out = fr.parse_metadata(batchname, batch_path,local_output_path, df_out)
    df_out = fr.handwritten_per(part,df_out)
    df_out = fr.check_stamp(df_out, files_with_stamp)



    # write to a tmp path on compute instance
    temp_out_file_path = local_output_path + 'OCS_Test_'+part +'_02102023'+'.csv'
    df_out.to_csv(temp_out_file_path, index = False)

    print('read and label batch {} successfully, save to {}'.format(part,temp_out_file_path))

Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


Read 2510 images' text in batch Feb_Part3
Labelled batch Feb_Part3
Calculating handwritten percentage of files in batch Feb_Part3
read and label batch Feb_Part3 successfully, save to output/OCS_Test_Feb_Part3_02102023.csv


In [17]:
df = pd.read_csv('output/Oct-25-2022_08000003-154055.csv')
df = fr.sort_document('111', df)
df.head()

Labelled batch 111


,file_name,content,word_confidence,styles,handwritten_percentage,has_stamp,page_label
0,Oct-25-2022_08000003-154055_00010.tif,OTIP Health Claims PO Box 280 Waterloo ON N2J ...,"{'OTIP': 0.961, 'Health': 0.997, 'Claims': 0.6...","[DocumentStyle(is_handwritten=True, spans=[Doc...",0.058,False,drug_receipt
1,Oct-25-2022_08000003-154055_00011.tif,SECTION 5: EQUIPMENT AND APPLIANCE EXPENSES Fo...,"{'SECTION': 0.994, '5:': 0.997, 'EQUIPMENT': 0...","[DocumentStyle(is_handwritten=True, spans=[Doc...",0.007,False,claim_page_two
2,Oct-25-2022_08000003-154055_00020.tif,Cleveland Clinic Canada James Anderson 820 Mil...,"{'Cleveland': 0.993, 'Clinic': 0.997, 'Canada'...","[DocumentStyle(is_handwritten=True, spans=[Doc...",0.003,False,paramedical invoice
3,Oct-25-2022_08000003-154055_00030.tif,Cleveland Clinic Canada James Anderson 820 Mil...,"{'Cleveland': 0.994, 'Clinic': 0.997, 'Canada'...","[DocumentStyle(is_handwritten=True, spans=[Doc...",0.003,False,paramedical invoice
4,Oct-25-2022_08000003-154055_00040.tif,Cleveland Clinic Canada James Anderson 820 Mil...,"{'Cleveland': 0.993, 'Clinic': 0.997, 'Canada'...","[DocumentStyle(is_handwritten=True, spans=[Doc...",0.003,False,paramedical invoice


In [18]:
df.to_csv('output/Oct-25-2022_08000003-154055.csv')

In [14]:
rules = fraud_document_analytics()

In [15]:
out_df = pd.DataFrame()
for files in os.listdir(local_output_path):
    if files.startswith('OCS') and files.endswith('02102023.csv'):
        doc_text_path = local_output_path+files
        print(doc_text_path)
        df = pd.read_csv(doc_text_path)
        invoices = ['paramedical invoice', 'other_receipt']
        df = df[df['page_label'].isin(invoices)]
        df = rules.paid_by_cash(df)
        df = rules.has_dollar_symbol(df)
        df = rules.handwritten_check(df,0.3)
        df = rules.invoice_num_check(df)
        df = rules.get_payment(df)
        df = rules.address_check(df)
        df = rules.get_reg_num(df)
    
        out_df = out_df.append(df, ignore_index = True)

out_df = out_df.fillna(False)

 #Remove Costco cases
for ind in out_df.index:
    if out_df['has_stamp'][ind] ==True and out_df['content'][ind].lower().find('costco')>-1:
        print(out_df['file_name'][ind])
        out_df['has_stamp'][ind] = False

output/OCS_Test_Feb_Part1_02102023.csv
output/OCS_Test_Feb_Part3_02102023.csv


In [38]:
out_df.head(20)

,file_name,content,word_confidence,styles,page_label,handwritten_percentage,has_stamp,paid_by_cash,no_dollar_symbol,above_handwritten_threshold,invalid_invoice_nbr,possible_invoice_nbr,payment_amount,has_invalid_addr,verified_address,register_num
0,OCS_files_Part_1_0050064385309_Step-_Element-_...,P: 204.560.5710 | F: 204.560.5715 1873 GRANT A...,"{'P:': 0.991, '204.560.5710': 0.98, '|': 0.065...",[],other_receipt,0.000,False,False,False,False,False,0000715g,240,False,False,False
1,OCS_files_Part_1_0050064385313_Step-_Element-_...,OFFICIAL RECEIPT Received from: Laura Leslie $...,"{'OFFICIAL': 0.995, 'RECEIPT': 0.996, 'Receive...","[DocumentStyle(is_handwritten=True, spans=[Doc...",paramedical invoice,0.181,False,False,False,False,False,False,False,False,False,False
2,OCS_files_Part_1_0050064385313_Step-_Element-_...,OFFICIAL RECEIPT Received from: Laura Leslie $...,"{'OFFICIAL': 0.994, 'RECEIPT': 0.996, 'Receive...","[DocumentStyle(is_handwritten=True, spans=[Doc...",paramedical invoice,0.181,False,False,False,False,False,False,False,False,False,False
3,OCS_files_Part_1_0050064385322_Step-_Element-_...,Invoice / Facture Paid / Payé Sold by / Vendu ...,"{'Invoice': 0.931, '/': 0.994, 'Facture': 0.99...",[],other_receipt,0.000,False,False,False,False,False,False,32.77,True,False,False
4,OCS_files_Part_1_0050064385323_Step-_Element-_...,TORONTO PSYCHOLOGY CLINIC Toronto Psychology C...,"{'TORONTO': 0.996, 'PSYCHOLOGY': 0.994, 'CLINI...",[],paramedical invoice,0.000,False,False,False,False,False,#20318-p01,180,False,False,False
5,OCS_files_Part_1_0050064385333_Step-_Element-_...,PHYSIOTHERAPY 880 Upper Wentworth St #2B Hamil...,"{'PHYSIOTHERAPY': 0.843, '880': 0.954, 'Upper'...",[],paramedical invoice,0.000,False,False,False,False,False,#75438,228,False,False,False
6,OCS_files_Part_1_0050064385335_Step-_Element-_...,PERFORMANCE INTEGRATED HEALTH Jesbinder Dhoot ...,"{'PERFORMANCE': 0.991, 'INTEGRATED': 0.993, 'H...",[],paramedical invoice,0.000,False,False,False,False,False,#294686-po1,63,False,False,False
7,OCS_files_Part_1_0050064385355_Step-_Element-_...,"Summit Sports and Health 2-704 Broadway Ave, S...","{'Summit': 0.994, 'Sports': 0.993, 'and': 0.99...",[],paramedical invoice,0.000,False,False,False,False,False,#55298-p01,90,False,False,False
8,OCS_files_Part_1_0050064385360_Step-_Element-_...,Page 1 of 1 Bill To: Manulife Financial P.O. B...,"{'Page': 0.991, '1': 0.994, 'of': 0.992, 'Bill...",[],paramedical invoice,0.000,False,False,False,False,False,False,100,False,False,False
9,OCS_files_Part_1_0050064385364_Step-_Element-_...,"Thursday, December 8, 2022 Dr. Robert Waybrant...","{'Thursday,': 0.993, 'December': 0.996, '8,': ...",[],paramedical invoice,0.000,False,False,False,False,False,False,5,False,False,False


In [47]:
out_df[out_df['file_name'] == 'OCS_files_Part_1_0050064389898_Step-_Element-_04844a2b-ec90-4f41-98d9-c96fbe193ede_3784E25D-55A4-40E7-A990-8AA127D519D4.jpeg.jpg']

,file_name,content,word_confidence,styles,page_label,handwritten_percentage,has_stamp,paid_by_cash,no_dollar_symbol,above_handwritten_threshold,invalid_invoice_nbr,possible_invoice_nbr,payment_amount,has_invalid_addr,verified_address,register_num,payment_above_200,payment_above_400,payment_above_500
1006,OCS_files_Part_1_0050064389898_Step-_Element-_...,COSTCO EOPTICAL INVOICE DATE / DATE DE FACTURE...,"{'COSTCO': 0.996, 'EOPTICAL': 0.937, 'INVOICE'...","[DocumentStyle(is_handwritten=True, spans=[Doc...",other_receipt,0.001,False,True,False,False,False,False,260,True,False,False,True,False,False


In [16]:
out_df['payment_above_200'] = np.where(out_df['payment_amount']>= 200, True, False)
out_df['payment_above_400'] = np.where(out_df['payment_amount']>= 400, True, False)
out_df['payment_above_500'] = np.where(out_df['payment_amount']>= 500, True, False)
out_df.loc[out_df["handwritten_percentage"] > 0.3 , "has_invalid_addr"] = False

df_to_rank =out_df[['file_name','has_stamp','paid_by_cash','no_dollar_symbol','above_handwritten_threshold','invalid_invoice_nbr','has_invalid_addr','payment_above_200','payment_above_400','payment_above_500']]
df_to_rank = df_to_rank.astype({'has_stamp': int, 'paid_by_cash': int, 'no_dollar_symbol':int,'above_handwritten_threshold':int,'invalid_invoice_nbr':int,'has_invalid_addr':int, 'payment_above_200':int, 'payment_above_400':int,'payment_above_500': int }) 


In [17]:
df_to_rank['paid_by_cash&payment_above'] = df_to_rank['paid_by_cash']*df_to_rank['payment_above_200']
df_to_rank['has_stamp&payment_above'] = df_to_rank['has_stamp']*df_to_rank['payment_above_500']
df_to_rank['no_dollar_symbol&payment_above'] = df_to_rank['no_dollar_symbol']*df_to_rank['payment_above_500']
df_to_rank['above_handwritten_threshold&payment_above'] = df_to_rank['above_handwritten_threshold']*df_to_rank['payment_above_500']
df_to_rank['invalid_invoice_nbr&payment_above'] = df_to_rank['invalid_invoice_nbr']*df_to_rank['payment_above_500']
df_to_rank['has_invalid_addr&payment_above'] = df_to_rank['has_invalid_addr']*df_to_rank['payment_above_400']


df_to_rank.loc[(df_to_rank['paid_by_cash&payment_above'] == 1), 'paid_by_cash'] = 0
df_to_rank.loc[(df_to_rank['has_stamp&payment_above'] == 1), 'has_stamp'] = 0
df_to_rank.loc[(df_to_rank['no_dollar_symbol&payment_above'] == 1), 'no_dollar_symbol'] = 0
df_to_rank.loc[(df_to_rank['above_handwritten_threshold&payment_above'] == 1), 'above_handwritten_threshold'] = 0
df_to_rank.loc[(df_to_rank['invalid_invoice_nbr&payment_above'] == 1), 'invalid_invoice_nbr'] = 0
df_to_rank.loc[(df_to_rank['has_invalid_addr&payment_above'] == 1), 'has_invalid_addr'] = 0

df_to_rank = df_to_rank.drop(columns=['payment_above_200','payment_above_400','payment_above_500'])

df_to_rank.rename(columns = {'paid_by_cash': 'paid_by_cash&payment_below'}, inplace = True)  
df_to_rank.rename(columns = {'has_stamp': 'has_stamp&payment_below'}, inplace = True)  
df_to_rank.rename(columns = {'no_dollar_symbol': 'no_dollar_symbol&payment_below'}, inplace = True)  
df_to_rank.rename(columns = {'above_handwritten_threshold': 'above_handwritten_threshold&payment_below'}, inplace = True)  
df_to_rank.rename(columns = {'invalid_invoice_nbr': 'invalid_invoice_nbr&payment_below'}, inplace = True)  
df_to_rank.rename(columns = {'has_invalid_addr': 'has_invalid_addr&payment_below'}, inplace = True)  

In [41]:
df_to_rank.head(20)

,file_name,has_stamp&payment_below,paid_by_cash&payment_below,no_dollar_symbol&payment_below,above_handwritten_threshold&payment_below,invalid_invoice_nbr&payment_below,has_invalid_addr&payment_below,paid_by_cash&payment_above,has_stamp&payment_above,no_dollar_symbol&payment_above,above_handwritten_threshold&payment_above,invalid_invoice_nbr&payment_above,has_invalid_addr&payment_above
0,OCS_files_Part_1_0050064385309_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
1,OCS_files_Part_1_0050064385313_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
2,OCS_files_Part_1_0050064385313_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
3,OCS_files_Part_1_0050064385322_Step-_Element-_...,0,0,0,0,0,1,0,0,0,0,0,0
4,OCS_files_Part_1_0050064385323_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
5,OCS_files_Part_1_0050064385333_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
6,OCS_files_Part_1_0050064385335_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
7,OCS_files_Part_1_0050064385355_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
8,OCS_files_Part_1_0050064385360_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0
9,OCS_files_Part_1_0050064385364_Step-_Element-_...,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
df_to_rank[(df_to_rank['has_invalid_addr']==1) & (df_to_rank['no_dollar_symbol']==1) &(df_to_rank['above_handwritten_threshold']==0)&(df_to_rank['paid_by_cash&payment_below']==0)&(df_to_rank['paid_by_cash&payment_above']==0)].count()

file_name                      154
has_stamp                      154
paid_by_cash&payment_below     154
no_dollar_symbol               154
above_handwritten_threshold    154
invalid_invoice_nbr            154
has_invalid_addr               154
paid_by_cash&payment_above     154
dtype: int64

In [27]:
df_to_rank.count()

file_name                                    4615
has_stamp&payment_below                      4615
paid_by_cash&payment_below                   4615
no_dollar_symbol&payment_below               4615
above_handwritten_threshold&payment_below    4615
invalid_invoice_nbr&payment_below            4615
has_invalid_addr&payment_below               4615
paid_by_cash&payment_above                   4615
has_stamp&payment_above                      4615
no_dollar_symbol&payment_above               4615
above_handwritten_threshold&payment_above    4615
invalid_invoice_nbr&payment_above            4615
has_invalid_addr&payment_above               4615
dtype: int64

In [18]:
weights = [0.0475,0.095,0.005,0.06,0.025,0.0175,0.38,0.19,0.02,0.24, 0.1, 0.07]

In [19]:
score = df_to_rank.drop(columns=['file_name']).mul(weights).sum(axis=1)

In [20]:
top_1 = score.quantile(0.99)
top_5 = score.quantile(0.95)

In [21]:
df_to_rank['risk_score'] = score

In [22]:
df_to_rank['risk_level'] = 'Not Suspicious'
df_to_rank.loc[df_to_rank.risk_score>=top_5,'risk_level'] = 'Should Investigate'
df_to_rank.loc[df_to_rank.risk_score>=top_1,'risk_level'] = 'Must Investigate'

In [23]:
df_to_rank.sort_values(by=['risk_score'], ascending = False).to_csv('OCS__risk_score_02102023.csv')

In [34]:
df_to_rank.sort_values(by=['risk_score'], ascending = False).head()

,file_name,has_stamp&payment_below,paid_by_cash&payment_below,no_dollar_symbol&payment_below,above_handwritten_threshold&payment_below,invalid_invoice_nbr&payment_below,has_invalid_addr&payment_below,paid_by_cash&payment_above,has_stamp&payment_above,no_dollar_symbol&payment_above,above_handwritten_threshold&payment_above,invalid_invoice_nbr&payment_above,has_invalid_addr&payment_above,risk_score,risk_level
1006,OCS_files_Part_1_0050064389898_Step-_Element-_...,0,1,0,0,0,1,1,0,0,0,0,0,0.4925,Must Investigate
677,OCS_files_Part_1_0050064388320_Step-_Element-_...,0,1,1,0,0,0,1,0,0,0,0,0,0.4800,Must Investigate
2869,OCS_files_Part_2_0050064380696_Step-_Element-_...,0,1,0,0,0,0,1,0,0,0,0,0,0.4750,Must Investigate
1047,OCS_files_Part_1_0050064390260_Step-_Element-_...,0,1,0,0,0,0,1,0,0,0,0,0,0.4750,Must Investigate
1091,OCS_files_Part_1_0050064390448_Step-_Element-_...,0,1,0,0,0,0,1,0,0,0,0,0,0.4750,Must Investigate


In [ ]:
df_to_rank.drop(columns=['file_name','risk_score','risk_level']).sum(axis=1)

In [36]:
df = df_to_rank.iloc[:,1:-2]
df[df.sum(axis=1) >= 3].count()

has_stamp                      6
paid_by_cash&payment_below     6
no_dollar_symbol               6
above_handwritten_threshold    6
invalid_invoice_nbr            6
has_invalid_addr               6
paid_by_cash&payment_above     6
dtype: int64

In [36]:
df_to_rank[df_to_rank['risk_score']>0].count()

file_name                                    1525
has_stamp&payment_below                      1525
paid_by_cash&payment_below                   1525
no_dollar_symbol&payment_below               1525
above_handwritten_threshold&payment_below    1525
invalid_invoice_nbr&payment_below            1525
has_invalid_addr&payment_below               1525
paid_by_cash&payment_above                   1525
has_stamp&payment_above                      1525
no_dollar_symbol&payment_above               1525
above_handwritten_threshold&payment_above    1525
invalid_invoice_nbr&payment_above            1525
has_invalid_addr&payment_above               1525
risk_score                                   1525
risk_level                                   1525
dtype: int64

In [27]:
df_to_rank.iloc[4871]

file_name                      01000001-122512_01910.tif
has_stamp                                              0
paid_by_cash&payment_below                             0
no_dollar_symbol                                       1
above_handwritten_threshold                            1
invalid_invoice_nbr                                    0
has_invalid_addr                                       1
paid_by_cash&payment_above                             0
risk_score                                          0.33
risk_level                              Must Investigate
Name: 4871, dtype: object

In [35]:
df_to_rank[df_to_rank['risk_level']=='Should Investigate'].count()

file_name                      285
has_stamp                      285
paid_by_cash&payment_below     285
no_dollar_symbol               285
above_handwritten_threshold    285
invalid_invoice_nbr            285
has_invalid_addr               285
paid_by_cash&payment_above     285
risk_score                     285
risk_level                     285
dtype: int64

In [33]:
df_to_rank[df_to_rank['invalid_invoice_nbr']==1].count()

file_name                      25
has_stamp                      25
paid_by_cash&payment_below     25
no_dollar_symbol               25
above_handwritten_threshold    25
invalid_invoice_nbr            25
has_invalid_addr               25
paid_by_cash&payment_above     25
risk_score                     25
risk_level                     25
dtype: int64

In [13]:
out_df = pd.DataFrame()
for files in os.listdir(local_output_path):
    doc_text_path = local_output_path+files
    if doc_text_path.endswith('csv'):
        print(doc_text_path)
        df = pd.read_csv(doc_text_path)
        for row in df.iterrows():
            text =str(row[1]['content']).lower()
            # print(text)
            if text.find('the physio spot') >-1:
                out_df = out_df.append(row[1], ignore_index = True)


output/Sept-2-2022_03000000-74657.csv
output/Sept-2-2022_03000001-081718.csv
output/Sept-2-2022_03000002-085127.csv
output/Sept-2-2022_03000006-111049.csv
output/Sept-2-2022_03000007-115700.csv
output/Sept-2-2022_03000008-120727.csv
output/Sept-20-2022_08000017-144934.csv
output/Sept-22-2022_08000000-75523.csv
output/Sept-22-2022_08000001-093417.csv
output/Sept-22-2022_08000002-103118.csv
output/Sept-22-2022_08000003-114545.csv
output/Sept-22-2022_08000005-133859.csv
output/Sept-22-2022_08000006-142703.csv
output/Sept-22-2022_08000007-145026.csv
output/Sept-22-2022_10000002-175400.csv
output/Sept-22-2022_10000003-180426.csv
output/Sept-26-2022_08000008-151315.csv
output/Sept-26-2022_08000009-154500.csv
output/Sept-27-2022_08000000-082728.csv
output/Sept-28-2022_08000000- 84023.csv
output/Sept-28-2022_08000001-090633.csv
output/Sept-28-2022_08000002-095510.csv
output/Sept-28-2022_08000003-112143.csv
output/Sept-28-2022_08000004-113154.csv
output/Sept-28-2022_08000005-120521.csv
output/S

In [25]:
df_to_rank.sort_values(by=['risk_score'], ascending = False)

,file_name,has_stamp,paid_by_cash&payment_below,no_dollar_symbol,above_handwritten_threshold,invalid_invoice_nbr,has_invalid_addr,paid_by_cash&payment_above,risk_score,risk_level
496,Oct-22-2022_10000004-132939_00570.tif,1,0,0,1,0,1,0,0.50,Must Investigate
526,Oct-22-2022_10000004-132939_01460.tif,0,0,0,0,0,0,1,0.38,Must Investigate
733,Oct-22-2022_10000007-123725_00720.tif,0,0,0,0,0,0,1,0.38,Must Investigate
114,Oct-19-2022_08000006-160654_02580.tif,0,0,1,1,0,1,0,0.33,Must Investigate
1098,Oct-24-2022_08000003-113921_01370.tif,0,0,1,1,0,1,0,0.33,Must Investigate
...,...,...,...,...,...,...,...,...,...,...
779,Oct-22-2022_10000007-123725_01810.tif,0,0,0,0,0,0,0,0.00,Not Suspicious
780,Oct-22-2022_10000007-123725_01820.tif,0,0,0,0,0,0,0,0.00,Not Suspicious
781,Oct-22-2022_10000007-123725_01830.tif,0,0,0,0,0,0,0,0.00,Not Suspicious
782,Oct-22-2022_10000007-123725_01840.tif,0,0,0,0,0,0,0,0.00,Not Suspicious
